In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import numpy as np

import warnings
warnings.filterwarnings('ignore')

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
data = pd.read_csv("IMDB Dataset.csv")

data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


# Предобработка данных

In [ ]:
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

def data_preprocessing(texts):
    cleaned_text = []
    for text in texts:
        text = text.lower()
        text = text.translate(str.maketrans('', '', string.punctuation)) # remove punctuation
        text = ''.join([i for i in text if not i.isdigit()]) # remove digits
        stop_words = set(stopwords.words('english'))
        words_list = [w for w in text.split() if w not in stop_words] # remove stop words
        words_list = [lemmatizer.lemmatize(w) for w in words_list] # lemmatization
        words_list = [stemmer.stem(w) for w in words_list] # stemming
        cleaned_text.append(' '.join(words_list))
    return cleaned_text

In [ ]:
cleaned_review = data_preprocessing(data['review'])

cleaned_data = data.copy()
cleaned_data['review'] = cleaned_review

In [ ]:
cleaned_data.head()

,review,sentiment
0,one review mention watch oz episod youll hook ...,positive
1,wonder littl product br br film techniqu unass...,positive
2,thought wonder way spend time hot summer weeke...,positive
3,basic there famili littl boy jake think there ...,negative
4,petter mattei love time money visual stun film...,positive


In [ ]:
cleaned_data['sentiment'].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(cleaned_data['review'], cleaned_data['sentiment'], test_size=0.2, random_state=42)

# Bag-of-Words

Bag-of-Words (BoW) — это простой метод, который представляет текст как набор слов и их частот. Этот метод не учитывает порядок слов, но позволяет быстро и эффективно преобразовать текст в числовой формат.

BoW метод хорошо подходит для задач, где важна частота слов, но не их порядок. Однако, он не учитывает семантические отношения между словами, что может быть недостатком для некоторых задач.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Преобразование данных в векторы
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(X_train)
X_train_count = vectorizer.transform(X_train)
X_test_count = vectorizer.transform(X_test)

# Обучение модели
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_count, y_train)

# Предсказание меток для тестовых данных
y_pred = rfc.predict(X_test_count)

# Оценка точности модели
accuracy_rf_bow = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_rf_bow:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.85
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      4961
    positive       0.85      0.86      0.86      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

[[4217  744]
 [ 718 4321]]


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Преобразование данных в векторы
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(X_train)
X_train_count = vectorizer.transform(X_train)
X_test_count = vectorizer.transform(X_test)

# Обучение модели
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_count, y_train)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_count)

# Оценка точности модели
accuracy_lr_bow = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_lr_bow:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.88
              precision    recall  f1-score   support

    negative       0.89      0.87      0.88      4961
    positive       0.88      0.89      0.88      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

[[4318  643]
 [ 537 4502]]


# TF-IDF

TF-IDF (Term Frequency-Inverse Document Frequency) учитывает не только частоту слов в документе, но и их значимость в корпусе текстов. Этот метод помогает уменьшить влияние часто встречающихся слов, которые не несут значимой информации.

TF-IDF метод позволяет более точно учитывать значимость слов в тексте, что улучшает качество анализа. Этот метод часто используется в задачах классификации текста и информационного поиска.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

## Random Forest

In [ ]:
# Преобразование данных в векторы TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_tfidf, y_train)

# Предсказание меток для тестовых данных
y_pred = rfc.predict(X_test_tfidf)

# Оценка точности модели
accuracy_rf_tfidf = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_rf_tfidf:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.85
              precision    recall  f1-score   support

    negative       0.85      0.85      0.85      4961
    positive       0.85      0.85      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

[[4221  740]
 [ 773 4266]]


## Logistic Regression

In [ ]:
# Преобразование данных в векторы TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_tfidf, y_train)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_tfidf)

# Оценка точности модели
accuracy_lr_tfidf = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_lr_tfidf:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.89
              precision    recall  f1-score   support

    negative       0.90      0.87      0.89      4961
    positive       0.88      0.91      0.89      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

[[4330  631]
 [ 471 4568]]


# Word Embeddings

Word Embeddings, такие как Word2Vec или GloVe, представляют слова в виде векторов в многомерном пространстве, что позволяет учитывать семантические отношения между словами. Эти методы позволяют моделям машинного обучения лучше понимать контекст и значение слов.

Word Embeddings методы позволяют учитывать контекст и семантические отношения между словами, что делает их более мощными для задач анализа текста. Эти методы часто используются в задачах, требующих глубокого понимания текста, таких как машинный перевод и генерация текста.

In [ ]:
from gensim.models import Word2Vec

## Random Forest

In [ ]:
# Обучение модели Word2Vec
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, sg=0)

# Преобразование данных в векторы
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])

# Обучение модели
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_w2v, y_train)

# Предсказание меток для тестовых данных
y_pred = rfc.predict(X_test_w2v)

# Оценка точности модели
accuracy_rf_w2v = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_rf_w2v:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.83
              precision    recall  f1-score   support

    negative       0.83      0.81      0.82      4961
    positive       0.82      0.84      0.83      5039

    accuracy                           0.83     10000
   macro avg       0.83      0.83      0.83     10000
weighted avg       0.83      0.83      0.83     10000

[[4018  943]
 [ 794 4245]]


## Logistic Regression

In [ ]:
# Обучение модели Word2Vec
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, sg=0)

# Преобразование данных в векторы
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])

# Обучение модели
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_w2v, y_train)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_w2v)

# Оценка точности модели
accuracy_lr_w2v = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_lr_w2v:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.85
              precision    recall  f1-score   support

    negative       0.86      0.84      0.85      4961
    positive       0.85      0.86      0.85      5039

    accuracy                           0.85     10000
   macro avg       0.85      0.85      0.85     10000
weighted avg       0.85      0.85      0.85     10000

[[4175  786]
 [ 696 4343]]


# Без предобработки данных

In [ ]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(data['review'], data['sentiment'], test_size=0.2, random_state=42)

## Bag-of-Words

### Random Forest

In [ ]:
# Преобразование данных в векторы
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(X_train)
X_train_count = vectorizer.transform(X_train)
X_test_count = vectorizer.transform(X_test)

# Обучение модели
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_count, y_train)

# Предсказание меток для тестовых данных
y_pred = rfc.predict(X_test_count)

# Оценка точности модели
accuracy_rf_bow_orig = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_rf_bow_orig:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.86
              precision    recall  f1-score   support

    negative       0.86      0.87      0.86      4961
    positive       0.87      0.86      0.86      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

[[4300  661]
 [ 710 4329]]


### Logistic Regression

In [ ]:
# Преобразование данных в векторы
vectorizer = CountVectorizer(stop_words='english')
vectorizer.fit(X_train)
X_train_count = vectorizer.transform(X_train)
X_test_count = vectorizer.transform(X_test)

# Обучение модели
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_count, y_train)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_count)

# Оценка точности модели
accuracy_lr_bow_orig = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_lr_bow_orig:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.88
              precision    recall  f1-score   support

    negative       0.89      0.88      0.88      4961
    positive       0.88      0.89      0.89      5039

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000

[[4357  604]
 [ 555 4484]]


## TF-IDF

### Random Forest

In [ ]:
# Преобразование данных в векторы TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_tfidf, y_train)

# Предсказание меток для тестовых данных
y_pred = rfc.predict(X_test_tfidf)

# Оценка точности модели
accuracy_rf_tfidf_orig = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_rf_tfidf_orig:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.86
              precision    recall  f1-score   support

    negative       0.85      0.87      0.86      4961
    positive       0.87      0.85      0.86      5039

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000

[[4306  655]
 [ 749 4290]]


### Logistic Regression

In [ ]:
# Преобразование данных в векторы TF-IDF
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf_vectorizer.fit(X_train)
X_train_tfidf = tfidf_vectorizer.transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

# Обучение модели
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_tfidf, y_train)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_tfidf)

# Оценка точности модели
accuracy_lr_tfidf_orig = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_lr_tfidf_orig:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.89
              precision    recall  f1-score   support

    negative       0.91      0.88      0.89      4961
    positive       0.88      0.91      0.90      5039

    accuracy                           0.89     10000
   macro avg       0.89      0.89      0.89     10000
weighted avg       0.89      0.89      0.89     10000

[[4352  609]
 [ 450 4589]]


## Word Embeddings

### Random Forest

In [ ]:
# Обучение модели Word2Vec
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, sg=0)

# Преобразование данных в векторы
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])

# Обучение модели
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train_w2v, y_train)

# Предсказание меток для тестовых данных
y_pred = rfc.predict(X_test_w2v)

# Оценка точности модели
accuracy_rf_w2v_orig = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_rf_w2v_orig:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.78
              precision    recall  f1-score   support

    negative       0.77      0.78      0.78      4961
    positive       0.78      0.78      0.78      5039

    accuracy                           0.78     10000
   macro avg       0.78      0.78      0.78     10000
weighted avg       0.78      0.78      0.78     10000

[[3854 1107]
 [1122 3917]]


### Logistic Regression

In [ ]:
# Обучение модели Word2Vec
sentences = [sentence.split() for sentence in X_train]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=5, sg=0)

# Преобразование данных в векторы
def vectorize(sentence):
    words = sentence.split()
    words_vecs = [w2v_model.wv[word] for word in words if word in w2v_model.wv]
    if len(words_vecs) == 0:
        return np.zeros(100)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

X_train_w2v = np.array([vectorize(sentence) for sentence in X_train])
X_test_w2v = np.array([vectorize(sentence) for sentence in X_test])

# Обучение модели
clf = LogisticRegression(max_iter=100)
clf.fit(X_train_w2v, y_train)

# Предсказание меток для тестовых данных
y_pred = clf.predict(X_test_w2v)

# Оценка точности модели
accuracy_lr_w2v_orig = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy: {accuracy_lr_w2v_orig:.2f}')

# Вывод отчета о классификации и матрицы ошибок
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.82
              precision    recall  f1-score   support

    negative       0.82      0.81      0.82      4961
    positive       0.82      0.83      0.82      5039

    accuracy                           0.82     10000
   macro avg       0.82      0.82      0.82     10000
weighted avg       0.82      0.82      0.82     10000

[[4037  924]
 [ 859 4180]]


# Total

In [ ]:
data = {
    'Model': ['Random Forest', 'Logistic Regression',
              'Random Forest without data preprocessing', 'Logistic Regression without data preprocessing'],
    'BoW': [accuracy_rf_bow, accuracy_lr_bow,
            accuracy_rf_bow_orig, accuracy_lr_bow_orig],
    'TF-IDF': [accuracy_rf_tfidf, accuracy_lr_tfidf,
               accuracy_rf_tfidf_orig, accuracy_lr_tfidf_orig],
    'W2V': [accuracy_rf_w2v, accuracy_lr_w2v,
            accuracy_rf_w2v_orig, accuracy_lr_w2v_orig]
}

models_accuracy = pd.DataFrame(data)
models_accuracy

,Model,BoW,TF-IDF,W2V
0,Random Forest,0.85,0.85,0.83
1,Logistic Regression,0.88,0.89,0.85
2,Random Forest without data preprocessing,0.86,0.86,0.78
3,Logistic Regression without data preprocessing,0.88,0.89,0.82
